Using the given python script to generate a cutout of NGC3379 and convert the inverse variance map to sigma map

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np

# -------------------------------------------------
# Input brick files (you must edit these paths)
# -------------------------------------------------
image_file  = "/mnt/c/Users/lenovo/Desktop/FRB-capstone/legacysurvey-1619p125-image-r.fits.fz"
invvar_file = "/mnt/c/Users/lenovo/Desktop/FRB-capstone/legacysurvey-1619p125-invvar-r.fits.fz"


# -------------------------------------------------
# Target coordinates: NGC 3379 (M105)
# -------------------------------------------------
center = SkyCoord(ra=161.9567 * u.deg, dec=12.5817 * u.deg, frame="icrs")

# Cutout size on the sky
size = (200 * u.arcsec, 200 * u.arcsec)  # adjust if needed

# Output filenames
out_sci   = "NGC3379_r_cutout.fits"
out_sigma = "NGC3379_r_sigma_cutout.fits"

# -------------------------------------------------
# Read the brick images (note: data stored in HDU 1)
# -------------------------------------------------
with fits.open(image_file) as hdul_img, fits.open(invvar_file) as hdul_inv:
    img_hdu = hdul_img[1]      # science image
    inv_hdu = hdul_inv[1]      # inverse variance image

    data_img = img_hdu.data
    data_inv = inv_hdu.data

    # WCS from the science header
    wcs_full = WCS(img_hdu.header)

    # -------------------------------------------------
    # 1. Make science and invvar cutouts
    # -------------------------------------------------
    cutout_img = Cutout2D(data_img, position=center, size=size, wcs=wcs_full)
    cutout_inv = Cutout2D(data_inv, position=center, size=size, wcs=wcs_full)

    # -------------------------------------------------
    # 2. Convert invvar → sigma map
    #    invvar = 1/sigma^2  => sigma = 1/sqrt(invvar)
    # -------------------------------------------------
    inv = cutout_inv.data
    sigma = np.full(inv.shape, 1e9, dtype=float) # safer default for zero invvar
    mask = inv > 0
    sigma[mask] = 1.0 / np.sqrt(inv[mask])

    # -------------------------------------------------
    # 3. Write output FITS files with WCS
    # -------------------------------------------------
    hdr_cut = cutout_img.wcs.to_header()

    fits.PrimaryHDU(data=cutout_img.data, header=hdr_cut).writeto(out_sci, overwrite=True)
    fits.PrimaryHDU(data=sigma, header=hdr_cut).writeto(out_sigma, overwrite=True)

print("Wrote:", out_sci)
print("Wrote:", out_sigma)

Using Sextractor to generate a binary fits file, with the information of isolated point stars and small image cutouts of these stars ( 35 by 35 ) pixels for now, which will be fed to psfex.

In [ ]:
# edited default.sex to correct arcsec pixel ratio, to increase signal noise ratio(detect thresh) and to change file type of output
# edited default.param to only contain values of interest and include 35by35 pixel cutouts for psfex
# had to bring default.conv and default.nnw to current directory
# ran sex legacysurvey-1619p125-image-r.fits.fz -c default.sex -CATALOG_NAME ForPSF.cat

We verify that the CLASS_STAR is upheld through this python script. 

In [ ]:
from astropy.io import fits
from astropy.table import Table
import numpy as np

# Define your strict cutoffs
MIN_SNR = 30.0
MAX_ELLIP = 0.2
MIN_CLASS_STAR = 0.8  # The guideline from your PDF

catalog_file = 'ForPSF.cat'
clean_file = 'clean_stars.cat'

print(f"--- VERIFICATION REPORT: {catalog_file} ---")

# Load the catalog (Data is usually in HDU 2 for LDAC)
try:
    t = Table.read(catalog_file, hdu=2)
except FileNotFoundError:
    print("Error: forcpsf.cat not found!")
    exit()

total_objects = len(t)
print(f"Total Objects Detected: {total_objects}")

# 1. Calculate Ellipticity from Elongation
# SExtractor ELONGATION = A/B.   Ellipticity = 1 - (1/Elongation)
# (Note: PSFEx definition varies slightly, but this is the standard proxy)
ellipticity = (t['ELONGATION'] - 1.0)/(1.0 + t['ELONGATION'])

# 2. Apply the 'Proxy' Filters (What PSFEx is doing)
psfex_mask = (t['SNR_WIN'] > MIN_SNR) & (ellipticity < MAX_ELLIP) & (t['FLAGS'] == 0)
psfex_survivors = t[psfex_mask]

print(f"\n[Step 1] Applying PSFEx Filters (Roundness + SNR + Isolation):")
print(f"   Survivors: {len(psfex_survivors)} / {total_objects}")

# 3. VERIFY: Do these survivors actually meet the CLASS_STAR > 0.8 requirement?
bad_survivors = psfex_survivors[psfex_survivors['CLASS_STAR'] < MIN_CLASS_STAR]
num_bad = len(bad_survivors)

print(f"\n[Step 2] Verification Check:")
print(f"   Of the {len(psfex_survivors)} stars PSFEx would pick...")
print(f"   {num_bad} of them have CLASS_STAR < {MIN_CLASS_STAR} (FAIL)")
print(f"   {len(psfex_survivors) - num_bad} of them have CLASS_STAR >= {MIN_CLASS_STAR} (PASS)")

if num_bad == 0:
    print("\n✅ SUCCESS: Your PSFEx settings perfectly match the CLASS_STAR guideline!")
else:
    print(f"\n⚠️  WARNING: Your settings let in {num_bad} impostors (Round objects that aren't stars).")

# 4. ENFORCE: Create a "Perfect" Catalog
# We filter by EVERYTHING: SNR + Ellipticity + FLAGS + CLASS_STAR
final_mask = psfex_mask & (t['CLASS_STAR'] >= MIN_CLASS_STAR)
clean_table = t[final_mask]

# Save as a new FITS_LDAC file
# We need to preserve the HDU structure for PSFEx to read it
""" hdul = fits.open(catalog_file)
hdul[2].data = clean_table.as_array() # Replace data with filtered version
hdul.writeto(clean_file, overwrite=True) """

print("-" * 50)
print(f"✅ CLEANED CATALOG SAVED: {clean_file}")
print(f"   Contains {len(clean_table)} guaranteed stars.")
print(f"   Action: Run 'psfex {clean_file}' to use this guaranteed sample.")
print("-" * 50)

--- VERIFICATION REPORT: ForPSF.cat ---
Error: forcpsf.cat not found!


NameError: name 't' is not defined

: 

We now use PSFEx to generate a model of the telescope blur ( Point Spread Function ), we hope to find. We attempt FLAG = 0.

In [ ]:
# Changed snr > 30 over 20, Max_ellp = .2 over .3, and set FLAG to only include perfect stars
# ran psfex ForPSF.cat
# Analyzed the various statistic fits files

We now attempt to generate parameters through galfit